In [1]:
import requests
import pandas as pd
import json
import mysql.connector
from datetime import datetime, timedelta
import time
import nvda_config

import twelvedata
from twelvedata import TDClient


In [2]:
# set up login
Cpassword = nvda_config.mysql_password
td = TDClient(apikey=nvda_config.twelve_api_nvda) 
base_url = 'https://api.twelvedata.com'


In [3]:
# Establish MySQL object
mydb = mysql.connector.connect(
    host="localhost",   
    user="root",         
    password=Cpassword,  
    database="nvdav1"   
)

mycursor = mydb.cursor()

In [6]:
# Date range for historical data (funky start date due to downloading first month as test batch)
end_date = datetime.now()- timedelta(days=30)
start_date = end_date - timedelta(days=395)  

# Twelve Data API parameters
symbol = "NVDA"
interval = "1min"
outputsize = 5000  

# Fetch data in chunks
current_date = start_date
requests_made = 0

while current_date < end_date:
    if requests_made >= 2:  # Check if 4 requests have been made
        time.sleep(60)  # Wait for 60 seconds (1 minute)
        requests_made = 0  # Reset the request counter

    chunk_end_date = current_date + timedelta(days=30)  # Fetch 30 days at a time

    # Fetch time series data and indicators
    time_series = td.time_series(
        symbol=symbol,
        interval=interval,
        outputsize=outputsize,
        start_date=current_date.strftime("%Y-%m-%d"),
        end_date=chunk_end_date.strftime("%Y-%m-%d"),
    ).with_macd().with_ema().with_rsi()
    
    # Convert to pandas Dataframes
    time_series_df = time_series.as_pandas()
    time_series_df = time_series_df.astype(str)

    # Prepare for database insertion
    time_series_data = []
    indicator_data = []

    # Process and align data
    for index, row in time_series_df.iterrows():
        # Extract the datetime string
        datetime_str = row.name.strftime("%Y-%m-%d %H:%M:%S")

        time_series_data.append(
            (
                symbol,
                datetime_str,
                row["open"],
                row["high"],
                row["low"],
                row["close"],
                row["volume"],
            )
        )
        if pd.notna(row["macd"]):  # Check if MACD is not NaN
            indicator_data.append(
                (
                    symbol,
                    datetime_str,
                    row["macd"],
                    row["macd_signal"],
                    row["macd_hist"],
                    row["ema"],
                    row["rsi"],
                )
            )


    # Insert tick data into MySQL
    sql_time_series = """
    INSERT INTO stock_ticks (symbol, datetime, open, high, low, close, volume)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    mycursor.executemany(sql_time_series, time_series_data)

    # Insert indicator data into MySQL
    sql_indicators = """
    INSERT INTO technical_indicators (symbol, bydatetime, macd, bysignal, byhistogram, ema, rsi)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    mycursor.executemany(sql_indicators, indicator_data)
    current_date = chunk_end_date
    requests_made += 1
 

mydb.commit()
print(mycursor.rowcount, "records inserted.")

KeyError: 'volume'